## 2. Machine Learning for Regression


In [ ]:
# %pip install pandas numpy

import pandas as pd
import numpy as np

## 2.2 Data preparation

In [ ]:
# data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'

In [ ]:
# !wget $data 

In [ ]:
df = pd.read_csv('data.csv')
df.head()  ##Look for header inconsistency

### Cleaning header (make it consistence)

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
df.columns  #.columns is a data structure of pandas, similar to series

### Cleaning values, let's take 'make' column as example

In [ ]:
df['make'].str.lower().str.replace(' ', '_')

### However, we need to apply this to all the column values, not only for make column. First detect the string column

In [ ]:
df.dtypes  ##List column's value type

### Get only string types, which is object

In [ ]:
df.dtypes == 'object'
df.dtypes[df.dtypes == 'object']  ## Get only object type column

In [ ]:
strings = list(df.dtypes[df.dtypes == 'object'].index) ## Get the index of the series and convert it to list 
strings

### Loop through all string columns AND clean the columns value, just like we did for make column earlier

In [ ]:
for col in strings:
    df[col] = df[col].str.lower().str.replace(' ', '_')
df.head()

In [ ]:
df.dtypes

## 2.3 Exploratory data analysis

Exploratory data analysis (EDA) is an essential step in the data analysis process. It involves summarizing and visualizing the main characteristics of a dataset to gain insights and identify patterns or trends. By summarizing, visualizing, and cleaning the data, researchers can uncover patterns, identify relationships, and make informed decisions

### Before doing EDA, let's look at each column and print some values

In [ ]:
for col in df.columns:
    print(col)
    print(df[col].head())
    print()  #new line

### Let's find unique value

In [ ]:
for col in df.columns:
    # print(col)
    # print(df[col].unique()[:5]) # print only first five unique value
    print(f"number of unique values in {col} column: {df[col].nunique()}")  #notice the nunique vs unique. n means count
    # print()

In [ ]:
df

### Distribution of price

In [ ]:
# %pip install seaborn matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

## Display plot in notebook
%matplotlib inline

In [ ]:
sns.histplot(df.msrp, bins=50)   #bins number of bars in histogram
plt.show()

From graph, we see that lots of cars are cheap and only few are expensive. That means it is LONG-TAIL Distribution (many prices in a small range, but a few prices in a wide range) we need to zoom in a bit to ignore the long tail with too less datapoints

In [ ]:
sns.histplot(df.msrp[df.msrp < 100000], bins=50)
plt.show()

This kind of distribution (long tail, and the peak) is not good for ML models, because this distribution will confuse them. There is a way to get rid of the long tail, by applying logarithm (it compresses large values while spreading out smaller ones) to the price. This results in more compact values.

In [ ]:
#np.log([0, 1,10,1000,100000])
# problem with logarithm is when we have a 0, because log(0) does not exist
#np.log([0 + 1, 1 + 1, 10 + 1, 1000 + 1, 100000 + 1])
# Output: array([ 0.        ,  0.69314718,  2.39789527,  6.90875478, 11.51293546])
# 
# In order to not always add 1 there is a NumPy function
#np.log1p([0, 1,10,1000,100000])
# Output: array([ 0.        ,  0.69314718,  2.39789527,  6.90875478, 11.51293546])

price_logs = np.log1p(df.msrp)
price_logs

In [ ]:
sns.histplot(price_logs, bins=50)

You can see the long tail is gone and you see a nice bell curve shape of a so called normal distribution, what is ideal for ML models. But still there is the strange peak. This could be the minimum price of $1,000 of the platform.

### Missing values

The sum function sums across columns and shows for each column how much missing values (Nan) are there. This information is important when training a model.

In [ ]:
df.isnull().sum()

## 2.4 Setting up the validation framework

Let's draw it

To validate the model, we take the dataset and split it into three parts (train-val-test / 60-20-20). 
This means that we train the model on the training dataset, check if it works fine on the validation dataset, and leave the test dataset for the end.
For each of these three parts, we create the feature matrix X and the target variable y (Xtrain, ytrain, Xval, yval, Xtest, ytest). 
So, what we need to do is calculate how much 20% is.

In [ ]:
len(df) # Number of records of the whole dataset

In [ ]:
int(len(df) * 0.2) #Calculate 20% of whole dataset

In [ ]:
#Splitting data set intto three 

n = len(df)
n_val = int(n * 0.2)
n_test = int(n * 0.2)
n_train = n - n_val - n_test

n, n_val+n_test+n_train

In [ ]:
n_val, n_test, n_train #sizes of our dataframes

In [ ]:
#df.iloc[[10, 0, 3, 5]]  #iloc used to select row number

In [ ]:
#Dataset has been split into three
df_train = df.iloc[:n_train]  #get data upto 7149
df_val = df.iloc[n_train:n_train+n_val] #get data from 7150 to 9531
df_test = df.iloc[n_train+n_val:] #get data from 9532 to rest

In [ ]:
df_train

In [ ]:
df_val

In [ ]:
df_test

After the divisions, there is one crucial problem, the sequential problem. That’s a problem when there is an order in the dataset. 
That means we need to shuffle, otherwise, there are BMWs only in one dataset

In [ ]:
idx = np.arange(n)
idx #See, the output is sequential, we need to shuffle

In [ ]:
# Let's shuffle. To make it reproducible, we may use seed
# np.random.seed(2)
np.random.shuffle(idx)
idx

Using this shuffled index we can create our shuffled datasets for training, validation and for testing.

In [ ]:
df_train = df.iloc[idx[:n_train]]  #select multiple columns from idx. It selects shuffled columns index for all rows in the DataFrame.
df_val = df.iloc[idx[n_train:n_train+n_val]]
df_test = df.iloc[idx[n_train+n_val:]]

Now there is no order in the index column so we can reset index and drop the old index column.

In [ ]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
df_train.head()

In [ ]:
len(df_train), len(df_val), len(df_test)

Remember that we should apply the log1p transformation to the price column to help the model perform well.

In [ ]:
y_train = np.log1p(df_train.msrp.values)
y_val = np.log1p(df_val.msrp.values)
y_test = np.log1p(df_test.msrp.values)

In [ ]:
len(y_train)

We should remove msrp values from dataframes (df_train, df_val, df_test) to make sure that, we don’t accidentally use it for training purposes. Since it has long tail distribution which will confuses the model

In [ ]:
del df_train['msrp']
del df_val['msrp']
del df_test['msrp']

## 2.5 Linear regression

Source: https://knowmledge.com/2023/09/20/ml-zoomcamp-2023-machine-learning-for-regression-part-4/

Explanation: https://chatgpt.com/share/67dcf911-8848-8012-b97e-2a72ac30eab3

Linear regression is a way to predict numbers based on some input information. For example, if we want to predict the **price of a car**, we can use details like **its age, mileage, and horsepower**.  

### The General Idea:  
- **X (Feature Matrix)** → All the details about many cars (like a table with rows and columns).  
- **y (Target)** → The price of each car.  
- **g (Model)** → The linear regression formula that learns the relationship between features and price.  

### Looking at One Car:  
Instead of looking at many cars, let’s focus on just **one car**.  
- **xi** → The details of that one car (e.g., age, mileage, horsepower).  
- **yi** → The price of that car.  
- **g(xi)** → The model’s prediction for that car’s price.  

So, the model takes **xi (car details)** and predicts **yi (price)** using a formula.

In [ ]:
df_train.iloc[9]  #Just taking one car

We take as an example the characteristic enging_hp, city_mpg, and popularity.

In [ ]:
# xi = [180, 11, 1851]  #Old value
xi = [230, 18, 3916]

That’s almost everything we need to implement g(xi) ~ yi:

xi = (180, 11, 1851) with i = 10

need to implement the function g(xi2,xi2, … , xin) ~ yi

In [ ]:
# in code this would look like --> this is what we want to implement
 
def g(xi):
    # do something and return the predicted price
    return 10000
 
g(xi)
# Output: 10000

This function g is still not very useful, because it always returns a fixed price. 

We need to implement the function g(xi) = w0 + w1xi1 + w2xi2 + w3xi3 with w0 as bias term and w1, w2, and w3 as weights. This formula can be written as

https://knowmledge.com/wp-content/uploads/2023/09/linregex-1.jpg


However, the formula of linear regression in python looks like this since index start from 0 instead of 1
https://knowmledge.com/wp-content/uploads/2023/09/linreggen-1.jpg

The following snippet shows the implementation of the g-function (renamed as linear_regression)

In [ ]:
def linear_regression(xi):
    n = len(xi)

    pred = w0

    for j in range(n):
        pred = pred + w[j] * xi[j]

    return pred

In [ ]:
# sample values for w0 and w and the given xi
xi = [230, 18, 3916]
w0 = 0
w = [1, 1, 1]

In [ ]:
linear_regression(xi)

In [ ]:
# try some other values
w0 = 7.17
w = [0.01, 0.04, 0.002]
linear_regression(xi)

#### What does this actually mean?

We’ve just implemented the formula as mentioned before with given values:

In [ ]:
7.17 + 230*0.01 + 18*0.04 + 3916*0.002

- w0 = 7.17 bias term = the prediction of a car, if we don’t know anything about this
- engine_hp: 230 * 0.01 that means in this case per 100 hp the price will increase by $1
- city_mpg: 18 * 0.04 that means analog to hp, the more gallons the higher the price will be
- popularity: 3916 * 0.002 analog, but it doesn’t seem that it’s affecting the price too much, so for every extra mention on twitter the car becomes just a little bit more expensive

There is still one important step to do. Because we logarithmized (log(y+1)) the price at the beginning, we now have to undo that. This gives us the predicted price in $.

#### Get the real prediction for the price in $
We do "-1" here to undo the "+1" inside the log. Shortcut for -1

In [ ]:
np.expm1(18.022)

Just for checking

In [ ]:
np.log1p(67120494.33915682)

## 2.6 Linear regression vector form

Source: https://knowmledge.com/2023/09/20/ml-zoomcamp-2023-machine-learning-for-regression-part-5/

Now we will generalize to a vector form of what we did in last. 

That means coming back from only one observation xi (of one car) to the whole feature matrix X.

https://knowmledge.com/wp-content/uploads/2023/09/linreggen-1.jpg

Looking at the last part of this formula we see the dot product (vector-vector multiplication).

g(xi) = w0 + xiTw

In [ ]:
# Let’s implement again the dot product (vector-vector multiplication)
def dot(xi, w):
    n = len(xi)
    
    res = 0.0
    
    for j in range(n):
        res = res + xi[j] * w[j]
    
    return res

Based on that the implementation of the linear_regression function could look like:

In [ ]:
def linear_regression(xi):
    return w0 + dot(xi, w)

To make the last equation more simple, we can imagine there is one more feature xi0, that is always equal to 1.

#g(xi) = w0 + xiTw -> g(xi) = w0xi0 + xiTw

That means we can use the dot product for the entire regression.

In [ ]:
# xi = [180, 11, 1851]  #old value
xi = [230, 18, 3916]
w0 = 7.17
w = [0.01, 0.04, 0.002]

In [ ]:
### adding w0 to the vector w
w_new = [w0] + w
w_new

In [ ]:
xi

The updated code for linear_regression function looks now like

In [ ]:
def linear_regression(xi):
    xi = [1] + xi
    return dot(xi, w_new)

In [ ]:
linear_regression(xi)

let’s go back to thinking about all the examples together. 

X is a m x (n+1) dimensional matrix (with m rows and n+1 columns)

https://knowmledge.com/wp-content/uploads/2023/09/xandw.jpg

What we have to do here, for each row of X we multiply this row with the vector w. 

This vector contains our predictions, therefor we call it ypred.

https://knowmledge.com/wp-content/uploads/2023/09/ypred.jpg

In [ ]:
# To sum up. What we need to do to get our model g is a matrix vector multiplication between X and w.
w0 = 7.17
w = [0.01, 0.04, 0.002]
w_new = [w0] + w

In [ ]:
x1  = [1, 148, 24, 1385]   #assume those are feature of bmw
x2  = [1, 132, 25, 2031]   #assume those are feature of toyota
x10 = [1, 453, 11, 86]     #assume those are feature of nissan

X = [x1, x2, x10]
X

In [ ]:
# This turns the list of lists into a matrix
X = np.array(X)
X

In [ ]:
# Now we have predictions, so for each car we have a price for this car
y = X.dot(w_new)
 
# shortcut to not do -1 manually to get the real $ price
np.expm1(y) 

In [ ]:
# The next snippet shows the implementation of the adapted linear_regression function
def linear_regression(X):
    return X.dot(w_new) #Maybe you wonder where the w_new vector comes from, we will talk it about in 2.7 section

In [ ]:
y = linear_regression(X)
np.expm1(y)

## 2.7 Training a linear regression model

Source: https://knowmledge.com/2023/09/21/ml-zoomcamp-2023-machine-learning-for-regression-part-6/


Simplify: https://chatgpt.com/share/67de481e-8728-8012-b196-18193a21d190

Now our goal is to compute the weight vector 𝑤 such that the prediction 𝑋𝑤 closely approximates 𝑦 in linear regression

However, directly solving Xw=y using w=X −1 y is not feasible because X −1 only exists for square matrices. However, Xis usually m×(n+1), which is not square.

In [ ]:
def train_linear_regression(X, y):
    pass

In [ ]:
# To approach this implementation we first use a simplified example.
X = [      
    [148, 24, 1385],
    [132, 25, 2031],
    [453, 11, 86],
    [158, 24, 185],
    [172, 25, 201],
    [413, 11, 86],
    [38,  54, 185],
    [142, 25, 431],
    [453, 31, 86],
]

X = np.array(X)
X

In [ ]:
# we need to add a new column with ones to the feature matrix X. That is for the multiplication with w0
ones = np.ones(9) #Creates a 1D array with 9 elements, and all the elements are 1
ones

In [ ]:
# X.shape[0] looks at the number of rows and creates the vector of ones
ones = np.ones(X.shape[0]) # Example: If X has 5 rows, it creates [1. 1. 1. 1. 1.].
ones

In [ ]:
# Now we need to stack this vector of ones with our feature matrix X
np.column_stack([ones, ones])

In [ ]:
X = np.column_stack([ones, X])

In [ ]:
y = [10000, 20000, 15000, 20050, 10000, 20000, 15000, 25000, 12000]

In [ ]:
# GRAM MATRIX
XTX = X.T.dot(X)
# Inverse GRAM MATRIX
XTX_inv = np.linalg.inv(XTX)

In the following code snippet we test whether the multiplication of XTX with XTX_inv actually produces the Identity matrix I.

In [ ]:
# Without round(1) it's not exactly identity matrix but the other values 
# are very close to 0 --> we can treat them as 0 and take it as identity matrix
XTX.dot(XTX_inv)

In [ ]:
# This gives us the I matrix
XTX.dot(XTX_inv).round(1)

In [ ]:
#Now we can implement the formula to get the full w vector.
w_full = XTX_inv.dot(X.T).dot(y)
w_full

From that vector w_full we can extract w0 and all the other weights.

In [ ]:
w0 = w_full[0]
w = w_full[1:]
w0, w

In [ ]:
#Now we can implement the function train_linear_regression, 
#that takes the feature matrix X and the target variable y and returns w0 and the vector w.
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

Let’s test this newly implemented function with some simple examples:

In [ ]:
X =[
    [148, 24, 1385],
    [132, 25, 2031],
    [453, 11, 86],
    [158, 24, 185],
    [172, 25, 201],
    [413, 11, 83],
    [38, 54, 185],
    [142, 25, 431],
    [453, 31, 86],  
]
 
X = np.array(X)
y = [10000, 20000, 15000, 25000, 10000, 20000, 15000, 25000, 12000]
 
train_linear_regression(X, y)

## 2.8 Car price baseline model

Source: https://knowmledge.com/2023/09/21/ml-zoomcamp-2023-machine-learning-for-regression-part-7/

Here we’ll use the implemented code from the last steps to build the model.
First we start with a simple model while we’re using only numerical columns.

The next code snippet shows how to extract all numerical columns. 

In [ ]:
df_train.dtypes

In [ ]:
df_train.columns

In [ ]:
# We choose the columns engine_hp, engine_cylinders, highway_mpg, city_mpg, and popularity for our base model.
base = ['engine_hp', 'engine_cylinders', 'highway_mpg',
        'city_mpg', 'popularity']

df_train[base].head()

In [ ]:
#We need to extract the values to use them in training.
X_train = df_train[base].fillna(0).values
X_train

In [ ]:
###Missing values are generally not good for our model. 
#Therefore, you should always check whether such values are present.
df_train[base].isnull().sum()

As you can see there are two columns (engine_hp & engine_cylinders) that have missing values. 
The easiest thing we can do is fill them with zeros. 
But notice filling it with 0 makes the model ignore this feature, because:

g(xi) = w0 + xi1w1 + xi2w2

if xi1 = 0 then the last equation simplifies to

g(xi) = w0 + 0 + xi2w2

But 0 is not always the best way to deal with missing values, because that means there is an observation of a car with 0 cylinders or 0 horse powers. 
And a car without cylinders or 0 horse powers does not make much sense at this point. 
For the current example this procedure is sufficient for us.


In [ ]:
df_train[base].fillna(0).isnull().sum()

In [ ]:
# However, now we need to apply this change in the DataFrame.
X_train = df_train[base].fillna(0).values
X_train

In [ ]:
y_train

Now we can train our model using the train_linear_regression function that we’ve implemented in the last article. 
The function return the value for w0 and and array for vector w.

In [ ]:
w0, w = train_linear_regression(X_train, y_train)
w0, w
# y_pred = w0 + X_train.dot(w)

We can use this two variables to apply the model to our training dataset to see how well the model has learned the training data.

In [ ]:
y_pred = w0 + X_train.dot(w)
y_pred

In [ ]:
### next snippet shows how to output these two lists accordingly.
# alpha changes the transparency of the bars
# bins specifies the number of bars
sns.histplot(y_pred, color='red', alpha=0.5, bins=50)
sns.histplot(y_train, color='blue', alpha=0.5, bins=50)
plt.show()

## 2.9 RMSE

Source: https://knowmledge.com/2023/09/22/ml-zoomcamp-2023-machine-learning-for-regression-part-8/

What we did here? Explanation is here: https://knowmledge.com/2023/09/22/ml-zoomcamp-2023-machine-learning-for-regression-part-8/

In [ ]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)

In [ ]:
# last article we used Seaborn to visualize the performance but now we have an objective metric for the evaluation.
rmse(y_train, y_pred)

## 2.10 Validating the model

Since we don’t know how well the model can apply the learned knowledge to unseen data. So what we want to do now after training the model g on our training dataset, we want to apply it on the validation dataset to see how it performs on unseen data. We use RMSE for validating the performance.

In [ ]:
base = ['engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'popularity']
 
X_train = df_train[base].fillna(0).values
 
w0, w = train_linear_regression(X_train, y_train)
 
y_pred = w0 + X_train.dot(w)

Next we implement the prepare_X function. The idea here to provide the same way of preparing the dataset regardless of whether it’s train set, validation set, or test set.

In [ ]:
def prepare_X(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

Now we can use this function when we prepare data for the training and for the validation as well. In the training part we only use training dataset to train the model. In the validation part we prepare the validation dataset the same way like before and apply the model. Lastly we compute the rmse.

In [ ]:
# Training part:
X_train = prepare_X(df_train)
w0, w = train_linear_regression(X_train, y_train)

# Validation part:
X_val = prepare_X(df_val)
y_pred = w0 + X_val.dot(w)

# Evaluation part:
rmse(y_val, y_pred)

When we compare the RMSE from training with the value from validation (0.73 vs. 0.75) we see that the model performs similarly well on the seen and unseen data. That is what we have hoped for.

## 2.11 Simple feature engineering

https://knowmledge.com/2023/09/22/ml-zoomcamp-2023-machine-learning-for-regression-part-9/

Suppose we want to develop a new feature based on the existing ones in the feature matrix X. Let’s assume we want to use the year information as an age information. Let’s assume further we have year 2017.

In [ ]:
2017 - df_train.year

We can add this new feature ‘age’ to our prepare_X function. What is one important remark here. 
It’s a good way to copy the dataframe inside prepare_X. Otherwise while using df you’ll modify the original data, what ist mostly not wanted.

In [ ]:
base = ['engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'popularity']
 
def prepare_X(df):
    df = df.copy()
     
    df['age'] = 2017 - df.year
    features = base + ['age']
     
    df_num = df[features]
    df_num = df_num.fillna(0)
    # extracting the Numpy array
    X = df_num.values
    return X
 
X_train = prepare_X(df_train)
X_train

The output of the last snippet shows a list of lists. Each list has 6 items, 5 numerical columns and our new ‘age’ column. Let’s train a new model and see how the model performs.

In [ ]:
X_train = prepare_X(df_train)
w0, w = train_linear_regression(X_train, y_train)

X_val = prepare_X(df_val)
y_pred = w0 + X_val.dot(w)
rmse(y_val, y_pred)

We can see an improvement. The rmse decreased from 0.748 to 0.516. The improvement in the rmse was clear. Let’s see if this improvement can be seen in the plots as well.

In [ ]:
sns.histplot(y_pred, label='prediction', color='red', alpha=0.5, bins=50)
sns.histplot(y_val, label='target', color='blue',  alpha=0.5, bins=50)
plt.legend()
plt.show()

Here, too, a clear improvement can be seen. Many car prices are predicted much better. But there is still space for improvement.

## 2.12 Categorical variables

Source: https://knowmledge.com/2023/09/23/ml-zoomcamp-2023-machine-learning-for-regression-part-10/

Categorical variables are variables that are categories (typically strings) Here: make, model, engine_fuel_type, transmission_type, driven_wheels, market_category, vehicle_size, vehicle_style 

In [ ]:
df_train.dtypes

But, there is one value that looks like numerical variable, but it isn’t. number_of_doors is not really a numerical number  (it's float).

In [ ]:
df_train.number_of_doors

In [ ]:
df_train.number_of_doors == 2 # Find cars which have 2 doors

Typical way of encoding such categorical variables is that we represent it with a bunch of binary columns – so called one-hot encoding. 

We can imitate this encoding by turning the booleans from the last snippet into integers (1 and 0) and creating a new variable for each number of doors.

In [ ]:
df_train['num_doors_2'] = (df_train.number_of_doors == 2).astype('int')
df_train['num_doors_3'] = (df_train.number_of_doors == 3).astype('int')
df_train['num_doors_4'] = (df_train.number_of_doors == 4).astype('int')

But we can do this easier with string replacement.

In [ ]:
'num_doors_%s' % 4
# Output: 'num_doors_4'
 
# With that replacement we can write a loop
for v in [2, 3, 4]:
    df_train['num_doors_%s' % v] = (df_train.number_of_doors == v).astype('int')


In [ ]:
# We delete this because we'll use another solution
for v in [2, 3, 4]:
    del df_train['num_doors_%s' % v]

In [ ]:
# Let’s use this string replacement method in our prepare_X function.
def prepare_X(df):
    df = df.copy()
    features = base.copy()
     
    df['age'] = 2017 - df.year
    features.append('age')
     
    for v in [2, 3, 4]:
        df['num_doors_%s' % v] = (df.number_of_doors == v).astype('int')
        features.append('num_doors_%s' % v)
     
    df_num = df[features]
    df_num = df_num.fillna(0)
    # extracting the Numpy array
    X = df_num.values
    return X
 
prepare_X(df_train)


In [ ]:
# Now we can check if the model performance has improved with the new features.
X_train = prepare_X(df_train)
w0, w = train_linear_regression(X_train, y_train)

X_val = prepare_X(df_val)
y_pred = w0 + X_val.dot(w)
rmse(y_val, y_pred)

We see in contrast to the last training with rmse of 0.514 there is only a slightly improvement (0.516 vs 0.514, lower is better), almost negligible. So the number of doors feature is not that useful. Maybe the ‘Make’ information is more useful. (our target is lowering the rmse)

In [ ]:
df.make.nunique()

In [ ]:
df.make

There are 48 unique values in the ‘Make’ column. That could be too much. Let’s look at the most popular ones.

In [ ]:
df.make.value_counts().head()

In [ ]:
# If we want to get the actual values, we use the index property
df.make.value_counts().head().index

In [ ]:
# Wrap it in a usual Python list
makes = list(df.make.value_counts().head().index)
makes

In [ ]:
# We can now adapt again our prepare_X function to add the new feature.
def prepare_X(df):
    df = df.copy()
    features = base.copy()
     
    df['age'] = 2017 - df.year
    features.append('age')
     
    for v in [2, 3, 4]:
        df['num_doors_%s' % v] = (df.number_of_doors == v).astype('int')
        features.append('num_doors_%s' % v)
         
    for v in makes:
        df['make_%s' % v] = (df.make == v).astype('int')
        features.append('make_%s' % v)
     
    df_num = df[features]
    df_num = df_num.fillna(0)
    # extracting the Numpy array
    X = df_num.values
    return X



In [ ]:
X_train = prepare_X(df_train)
w0, w = train_linear_regression(X_train, y_train)
 
X_val = prepare_X(df_val)
y_pred = w0 + X_val.dot(w)
 
rmse(y_val, y_pred)

The model performance has once again improved somewhat (lowered from 0.514 to 0.507) How about adding all the other categorical variables now? This should improve the performance even more, right? Let’s try.

In [ ]:
categorical_variables = [
    'make', 'engine_fuel_type', 'transmission_type', 'driven_wheels', 
    'market_category', 'vehicle_size', 'vehicle_style'
]
 
# The dictionary category will contain for each of the categories 
# the top 5 most common ones
categories = {}
 
for c in categorical_variables:
    categories[c] = list(df[c].value_counts().head().index)
     
categories

In [ ]:
# The next snippet shows how to implement the new features to our prepare_X function. This time we need two loops as described inline.

def prepare_X(df):
    # this is good way to do, otherwise while using df you'll modify the original data
    # what is mostly not wanted
    df = df.copy()
    features = base.copy()
     
    df['age'] = 2017 - df.year
    features.append('age')
     
    for v in [2, 3, 4]:
        df['num_doors_%s' % v] = (df.number_of_doors == v).astype('int')
        features.append('num_doors_%s' % v)
 
    # First loop is for each key of the dictionary categories.
    # Second loop is for each value inside the categories
    # For each of this values we create a new column.
    for c, values in categories.items():    
        for v in values:
            df['%s_%s' % (c, v)] = (df[c] == v).astype('int')
            features.append('%s_%s' % (c, v))
     
    df_num = df[features]
    df_num = df_num.fillna(0)
    # extracting the Numpy array
    X = df_num.values
    return X

In [ ]:
X_train = prepare_X(df_train)
w0, w = train_linear_regression(X_train, y_train)
 
X_val = prepare_X(df_val)
y_pred = w0 + X_val.dot(w)
 
rmse(y_val, y_pred)

This time the model performance is very bad. As you can see the RMSE (3.1033420378110214e) is very large. So something went wrong. We will see in next

## 2.13 Regularization

In [ ]:
# Source: https://knowmledge.com/2023/09/23/ml-zoomcamp-2023-machine-learning-for-regression-part-11/

The topic for this part is regularization as a way to solve the problem of duplicated columns in our data. Remember the formula for normal equation is: w = (X^TX)^-1*X^Ty
The problem what we have is connected with the first part (X^TX)^-1. We need to take an inverse of the GRAM matrix. Sometimes this inverse doesn’t exist. This happens when there are duplicate features in X. Let's see an example

In [ ]:
# # You see here 2nd and 3rd columns are identical
X = [
    [4, 4, 4],
    [3, 5, 5],
    [5, 1, 1],
    [5, 4, 4],
    [7, 5, 5],
    [4, 5, 5]
]

X = np.array(X)
X

In [ ]:
XTX = X.T.dot(X)
XTX

In [ ]:
# np.linalg.inv(XTX) 

The code from the last article didn’t raise that error, so the inverse of that gram matrix exists. But the reason for the very big value for rmse is that our data is not very clean.

Let’s go back to our last example but this time similar X as before with a few noise.

In [ ]:
X = [
    [4, 4, 4],
    [3, 5, 5],
    [5, 1, 1],
    [5, 4, 4],
    [7, 5, 5],
    [4, 5, 5.0000001],   #added little noise, 5.0000001 instead of 5
]
 
X = np.array(X)
y = [1, 2, 3, 1, 2, 3]
 
XTX = X.T.dot(X)
XTX
 
XTX_inv = np.linalg.inv(XTX)
XTX_inv

In [ ]:
w = XTX_inv.dot(X.T).dot(y)
w

In [ ]:
# Adding a small number to the diagonal
# helps to control. So the numbers of w become smaller
XTX = [
    [1.0001, 2, 2],
    [2,     1.0001, 1.0000001],
    [2, 1.0000001, 1.0001]
]

XTX = np.array(XTX)
np.linalg.inv(XTX)

In [ ]:
XTX = [
    [1.01, 2, 2],
    [2,   1.01, 1.0000001],
    [2, 1.0000001,   1.01]
]
 
XTX = np.array(XTX)
np.linalg.inv(XTX)

In [ ]:
XTX = [
    [1, 2, 2],
    [2, 1, 1.0000001],
    [2, 1.0000001, 1]
]
 
XTX =  np.array(XTX)
XTX

In [ ]:
# Remember there was the eye function to get an Identity matrix. Maybe we can use this…
np.eye(3)

In [ ]:
# When adding XTX to this matrix, it adds one on the diagonal
XTX + np.eye(3)

In [ ]:
# We can multiply this eye by a small number
XTX = XTX + 0.01 * np.eye(3)
XTX

In [ ]:
# XTX = XTX + 0.1*np.eye(3)
# XTX

In [ ]:
# XTX = XTX + 1*np.eye(3)
# XTX

In [ ]:
np.linalg.inv(XTX)

In [ ]:
### This leads us to reimplementing the train_linear_regression function again
# reg = regularized
# parameter r = short for regularization
def train_linear_regression_reg(X, y, r=0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0])

    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)
    
    return w_full[0], w_full[1:]

In [ ]:
X_train = prepare_X(df_train)
w0, w = train_linear_regression_reg(X_train, y_train, r=0.01)

X_val = prepare_X(df_val)
y_pred = w0 + X_val.dot(w)
rmse(y_val, y_pred)

## 2.14 Tuning the model

Source: https://knowmledge.com/2023/09/24/ml-zoomcamp-2023-machine-learning-for-regression-part-12/

In [ ]:
for r in [0.0, 0.00001, 0.0001, 0.001, 0.1, 1, 10]:
    X_train = prepare_X(df_train)
    w0, w = train_linear_regression_reg(X_train, y_train, r=r)

    X_val = prepare_X(df_val)
    y_pred = w0 + X_val.dot(w)
    score = rmse(y_val, y_pred)
    
    print("reg parameter: ",r, "bias term: ",w0, "rmse: ",score)

In [ ]:
r = 0.001
X_train = prepare_X(df_train)
w0, w = train_linear_regression_reg(X_train, y_train, r=r)

X_val = prepare_X(df_val)
y_pred = w0 + X_val.dot(w)
score = rmse(y_val, y_pred)
print("rmse: ",score)

## 2.15 Using the model

### Combining datasets
First step to do is getting our data. So we need to combine df_train and df_val into one dataset. We can use Pandas concat() function that takes a list of dataframes and concatenates them together.

In [ ]:
df_full_train = pd.concat([df_train, df_val])

We also need to concatenate y_train and y_val to get y_full_train. This time we use the concatenate function of NumPy library.

In [ ]:
y_full_train = np.concatenate([y_train, y_val])

In [ ]:
y_full_train

### Resetting index
When combining two dataframes it can happen that the index is not sequential. Here you can use an already known function and reset the index.

In [ ]:
df_full_train = df_full_train.reset_index(drop=True)

### Getting feature matrix X
Now we have again a coherent dataset for training and we can prepare it for the usage as we did before. The prepare_X() function still works fine.

In [ ]:
X_full_train = prepare_X(df_full_train)

In [ ]:
X_full_train

### Train the final model
Next step is to train the final model on the combined dataset. We’re using the new train_linear_regression_reg() function to get the value for w0 and the vector w.

In [ ]:
w0, w = train_linear_regression_reg(X_full_train, y_full_train, r=0.001)
w0, w

### Applying model to test data
Now is the great moment for the final model. It must pass the final test. For this purpose we use test data, which are again prepared with the prepare_X() function. Then the model is applied to the test data and the RMSE can be calculated.

In [ ]:
X_test = prepare_X(df_test)
y_pred = w0 + X_test.dot(w)

score = rmse(y_test, y_pred)

print("rmse: ", score)

RMSE_test = 0.4629526324225134 is not so far away from RMSE_val = 0.4549809533999964. That means the model generalizes quite well and it didn’t get this score by chance. Now we have our final model and we can use it. The way we want to use it is to predict the price of an (unseen) car – unseen means here that the model hasn’t seen this car during training.

### Using the model
Using the model means:

- Extracting all the features (getting feature vector of the car)
- Applying our final model to this feature vector & predicting the price

#### Feature Extraction
For this step we can take any car from our test dataset and pretend it’s a new car. Let’s just take one car.

In [ ]:
df_test.iloc[20]

Usually the way we do it is that we don’t get a dataframe here. But it could be a Python dictionary with all the information about the car. In real life you can imagine a website or an app, where people enter all the values. Then the website sends the request with all the information (as dictionary) to the model. The model replies back with the predicted price.

In [ ]:
# For this example we turn this data of our car into a dictionary.
car = df_test.iloc[20].to_dict()
car

In [ ]:
# The car is our request and now remember the prepare_X function expects a dataframe, so we need to create a dataframe with a single row for our request.
df_small = pd.DataFrame([car])
df_small

In [ ]:
# We can use this single row DataFrame as input for the prepare_X() function to get the feature matrix. In this case our feature matrix is a feature vector.
X_small = prepare_X(df_small)

#### Predicting the price
The final step is to apply the final model to our requested car (feature vector) and predict the price.

In [ ]:
y_pred = w0 + X_small.dot(w)
# Don't need an array but it's first (and only) item
y_pred = y_pred[0]
y_pred

10.67 is still not the price in $. To get the real price we need to undo the logarithm.

In [ ]:
np.expm1(y_pred)

After undoing the logarithm we get the price in $. 

So we think that a car with these characteristics should cost  $43,432.70.

In [ ]:
# Lastly to get an evaluation about model performance let’s compare the predicted price to the actual price of this requested car
np.expm1(y_test[20])

In [ ]:
# Original price is $50,549.99; not bad prediction comparatively $30,973.55 

## 2.16 Next steps

* We included only 5 top features. What happens if we include 10?

Other projects

* Predict the price of a house - e.g. boston dataset
* https://archive.ics.uci.edu/ml/datasets.php?task=reg
* https://archive.ics.uci.edu/ml/datasets/Student+Performance

## 2.17 Summary

* EDA - looking at data, finding missing values
* Target variable distribution - long tail => bell shaped curve
* Validation framework: train/val/test split (helped us detect problems)
* Normal equation - not magic, but math
* Implemented it with numpy
* RMSE to validate our model
* Feature engineering: age, categorical features
* Regularization to fight numerical instability